## prepare data

In [ ]:
import os
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import preprocessing

#%cd /content/vietnamese_news_ml_dl
# training data: 
path = r'/content/vietnamese_news_ml_dl/10topics_update_final/train_0.8'
X_train10 = []
Y_train10 = []
# for train10_origin
for topic in os.listdir(path):
  pd = os.path.join(path, topic)
  for file in os.listdir(pd):
    pf = os.path.join(pd, file)
    text = open(pf, 'r').read()
    # words = text.split()
    
    text = unidecode(text) # test sử dụng câu bỏ dấu

    X_train10.append(text)
    Y_train10.append(topic)

# test data: 
path = r'/content/vietnamese_news_ml_dl/10topics_update_final/test10_origin_processed'
X_test10 = []
Y_test10 = []
# for train10_origin
for topic in os.listdir(path):
  pd = os.path.join(path, topic)
  for file in os.listdir(pd):
    pf = os.path.join(pd, file)
    text = open(pf, 'r').read()
    # words = text.split()
  
    text = unidecode(text) # test sử dụng câu bỏ dấu

    X_test10.append(text)
    Y_test10.append(topic)

#valid data
path = r'/content/vietnamese_news_ml_dl/10topics_update_final/valid_0.8'
X_valid10 = []
Y_valid10 = []
for topic in os.listdir(path):
  pd = os.path.join(path, topic)
  for file in os.listdir(pd):
    pf = os.path.join(pd, file)
    text = open(pf, 'r').read()
    # words = text.split()
  
    text = unidecode(text) # test sử dụng câu bỏ dấu

    X_valid10.append(text)
    Y_valid10.append(topic)

count_vector = CountVectorizer()
X_train_tranform10 = count_vector.fit(X_train10)
X_train_tranform10 = count_vector.transform(X_train10)
# label encoder
le = preprocessing.LabelEncoder()
le.fit(Y_train10)
Y_train_transform10 = le.transform(Y_train10)

#for test data
X_valid_transform10 = count_vector.transform(X_valid10)
Y_valid_transform10 = le.transform(Y_valid10)

# for test data
X_test_transform10 = count_vector.transform(X_test10)
Y_test_transform10 = le.transform(Y_test10)




In [ ]:
# for Tfidf 
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_tranform10)
X_train10_tf = tf_transformer.transform(X_train_tranform10)
X_valid10_tf = tf_transformer.transform(X_valid_transform10)
X_test10_tf = tf_transformer.transform(X_test_transform10)

### Turning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time
for x in range(15,30):
  start = time.time()
  model_lgr_count = LogisticRegression(random_state=0, max_iter=x)
  model_lgr_count.fit(X_tranform10, Y_transform10)
  end = time.time()
  y_predict_train_lgr = model_lgr_count.predict(X_tranform10)
  y_predict_valid_lgr = model_lgr_count.predict(X_valid_transform10)
  print("inter = " ,x)
  print("acc per train :" , accuracy_score(y_predict_train_lgr, Y_transform10))
  print("acc per valid :" , accuracy_score(y_predict_valid_lgr,Y_valid_transform10))
  print("training time: ", end - start)

In [ ]:
for x in range(20,60):
  start = time.time()
  model_lgr_count = LogisticRegression(random_state=0, max_iter=x)
  model_lgr_count.fit(X_train10_tf, Y_transform10)
  end = time.time()
  y_predict_train_lgr = model_lgr_count.predict(X_train10_tf)
  y_predict_valid_lgr = model_lgr_count.predict(X_valid10_tf)
  print("inter = " ,x)
  print("acc per train :" , accuracy_score(y_predict_train_lgr, Y_transform10))
  print("acc per valid :" , accuracy_score(y_predict_valid_lgr,Y_valid_transform10))
  print("training time: ", end - start)

### Test and save Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import time
start = time.time()
model_lgr_count = LogisticRegression(random_state=0, max_iter=22)
model_lgr_count.fit(X_tranform10, Y_transform10)
end = time.time()

y_predict_train_lgr = model_lgr_count.predict(X_tranform10)
y_predict_valid_lgr = model_lgr_count.predict(X_valid_transform10)

print("acc per train :" , accuracy_score(y_predict_train_lgr, Y_transform10))
print("acc per valid :" , accuracy_score(y_predict_valid_lgr,Y_valid_transform10))
print("training time: ", end - start)

confusion_matrix(y_predict_test_lgr,Y_test_transform10)

In [ ]:
from joblib import dump, load
dump(model_lgr_count,"logisticsRegression.joblib")

### Kflod

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

num_folds = 10
acc_per_fold = []
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
for train, valid in kfold.split(X_tranform10,Y_transform10):
  metric = []
  model_lgr_count = LogisticRegression(random_state=0, max_iter=22)
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  model_lgr_count.fit(X_tranform10[train],Y_transform10[train])

  y_predict_train_lgr = model_lgr_count.predict(X_tranform10[train])
  score1 = accuracy_score(Y_transform10[train], y_predict_train_lgr)

  y_predict_valid_lgr = model_lgr_count.predict(X_tranform10[valid])
  score2 = accuracy_score(Y_transform10[valid],y_predict_valid_lgr)

  y_predict_test_lgr = model_lgr_count.predict(X_test_transform10)
  score3 = accuracy_score(Y_test_transform10,y_predict_test_lgr)
  metric.append([score1,score2,score3])
  print(f'Score for fold {fold_no}: on train: {score1}    on valid: {score2}    on test: {score3}')
  acc_per_fold.append(metric)
  fold_no += 1

acc_per_fold